In [60]:
import torch
import os
import torchvision.utils as vutils
import torch.nn as nn
import torch.nn.functional as F
from models import weights_init, Discriminator, Generator
from operation import copy_G_params, load_params

In [61]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

model_path = os.path.abspath("all_40000/all_40000.pth")  # 確保這個檔案存在

if not os.path.exists(model_path):
    raise FileNotFoundError(f"模型權重檔案不存在: {model_path}")


# latent_dim = 100  # 你的 Generator 使用的是 nz=100
netG = Generator(im_size=256, nz=256)
netG.apply(weights_init)

netD = Discriminator(im_size=256)
netD.apply(weights_init)

netG.to(device)
netD.to(device)

ckpt = torch.load(model_path, map_location=device, weights_only=True)
netG.load_state_dict(ckpt['g'])
netD.load_state_dict(ckpt['d'])
avg_param_G = ckpt['g_ema']
load_params(netG, avg_param_G)

cuda


In [62]:
for name, layer in netG.named_children():
    print(name)

init
feat_8
feat_16
feat_32
feat_64
feat_128
feat_256
se_64
se_128
se_256
to_128
to_big


In [63]:
import torch.optim as optim

# targets = [ 'se_64', 'se_128', 'se_256', 'to_big']
targets = ['init', 'feat_8', 'feat_16']
for name, layer in netG.named_children():
    for param in layer.parameters():
        if(name in targets):
            param.requires_grad = False
        else:
            param.requires_grad = True

nlr = 0.0002
nbeta1 = 0.0
weight_decay = 0.0003
g_optimizer = optim.Adam(netG.parameters(), lr=nlr, betas=(nbeta1, 0.99), weight_decay=weight_decay)
d_optimizer = optim.Adam(netD.parameters(), lr=nlr, betas=(nbeta1, 0.99), weight_decay=weight_decay)

In [ ]:
from operation import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data.dataloader import DataLoader
from operation import ImageFolder, InfiniteSamplerWrapper


def enhance_channel(img_tensor, channel_idx, factor=1.5):
    enhanced = img_tensor.clone()
    enhanced[channel_idx] = torch.clamp(enhanced[channel_idx] * factor, -1.0, 1.0)  
    print(type(enhanced))
    return enhanced

def color_augmentation(img_tensor):
    return [enhance_channel(img_tensor, i) for i in range(3)]

data_root = 'C:\\Users\\b4088\OneDrive\\Desktop\\DeepLearning\\Assignment\\FinalProject\\learn_style'
transform_list = [
            transforms.Resize((int(256),int(256))),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
            # transforms.Lambda(color_augmentation) 
        ]
trans = transforms.Compose(transform_list)

dataset = ImageFolder(root=data_root, transform=trans)
print(len(dataset))
dataloader = iter(DataLoader(dataset, batch_size=4, shuffle=False,
                      sampler=InfiniteSamplerWrapper(dataset), num_workers=2, pin_memory=True))


3


<>:16: SyntaxWarning: invalid escape sequence '\O'
<>:16: SyntaxWarning: invalid escape sequence '\O'
C:\Users\b4088\AppData\Local\Temp\ipykernel_33200\4014493556.py:16: SyntaxWarning: invalid escape sequence '\O'
  data_root = 'C:\\Users\\b4088\OneDrive\\Desktop\\DeepLearning\\Assignment\\FinalProject\\learn_style'


In [74]:
from tqdm import tqdm
from train import train_d
from diffaug import DiffAugment
policy = 'color,translation'


for i in tqdm(range(0, 120)):
    real_images = next(dataloader)
    real_images = real_images.to(device)
    real_images = DiffAugment(real_images, policy=policy)

    batch_size = real_images.size(0)

    z = torch.randn(batch_size, 256, 1, 1).to(device)

    fake_images = netG(z)
    fake_images = [DiffAugment(fake, policy=policy) for fake in fake_images]


    # train disc
    netD.zero_grad()

    err_dr, rec_img_all, rec_img_small, rec_img_part = train_d(netD, real_images, label="real")
    train_d(netD, [fi.detach() for fi in fake_images], label="fake")
    d_optimizer.step()

    # train gen
    netG.zero_grad()
    pred_g = netD(fake_images, "fake")
    err_g = -pred_g.mean()

    err_g.backward()
    g_optimizer.step()

    # real_logits = netD(real_images, label="real", part=0)
    # fake_logits = netD(fake_images.detach(), label="fake", part=0 )
    
    # print(fake_logits)
    # highr logits means discriminator believes it's a real image
    # d_loss = F.softplus(-real_logits[0]).mean() + F.softplus(fake_logits[0]).mean()
    
    # d_optimizer.zero_grad()
    # d_loss.backward()
    # d_optimizer.step()

    # fake_logits = netD(fake_images, label="fake", part=0)
    # g_loss = -F.softplus(fake_logits).mean()
    # g_loss = -fake_logits.mean()
    # g_optimizer.zero_grad()
    # g_loss.backward()
    # g_optimizer.step()
    # print(f"{i}, G Loss: {g_loss:.4f}, D Loss: {d_loss:.4f}")





  0%|          | 0/120 [00:05<?, ?it/s]


RuntimeError: DataLoader worker (pid(s) 5112, 19444) exited unexpectedly

In [ ]:
torch.save({'g': netG.state_dict(),'g_ema': netG.state_dict(), 'd': netD.state_dict()}, "fine_tuned_stylegan2.pth")

In [ ]:
import torch
import os
import torchvision.utils as vutils
import torch.nn as nn
import torch.nn.functional as F
from models import weights_init, Discriminator, Generator
from operation import copy_G_params, load_params


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model_path = os.path.abspath("models/all_50000.pth")  # 確保這個檔案存在
model_path = os.path.abspath("fine_tuned_stylegan2.pth")  # 確保這個檔案存在
if not os.path.exists(model_path):
    raise FileNotFoundError(f"模型權重檔案不存在: {model_path}")


# latent_dim = 100  # 你的 Generator 使用的是 nz=100
netG = Generator(im_size=256, nz=256)
netG.apply(weights_init)

netD = Discriminator(im_size=256)
netD.apply(weights_init)

netG.to(device)
netD.to(device)

ckpt = torch.load(model_path, map_location=device, weights_only=True)
netG.load_state_dict(ckpt['g'])
netD.load_state_dict(ckpt['d'])
# avg_param_G = ckpt['g_ema']
# load_params(netG, avg_param_G)


num_samples = 4  # 產生 4 張圖片
z = torch.randn(num_samples, 256).to(device)  


with torch.no_grad():
    generated_images = netG(z)[0]

generated_images = (generated_images + 1) / 2  # 轉換到 [0,1] 範圍
for i, img in enumerate(generated_images):
    vutils.save_image(img, f"generated_{i}.png")

print("✅ 圖片已生成並儲存！")


✅ 圖片已生成並儲存！
